In this notebook, we inspect **in which way a tabular dataset as Census can be used by an AI based on graphs to estimate wealthiness of individuals**. 

Therefore, we proceed in 2 steps:

**1. We prepare data to be handled by a model based on a graph**
We transform them into a graph, that involves strong assumptions on the features involved in connections...

**2. We train an AI based on graphs**
Here, we begin with a Graphical Neural Network (GNN) based on a Multi-Layer Perceptron (MLP), requiring the library Torch.

**3. We inspect if the graph-based AI indeed reflects common & expert knowledge on**
In particular, regarding the non-sense of certain inferences that should absolutely be avoided (e.g. education may influence occupation, but not the reverse).

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Data preparation for binary classification with graphs (Census)
For this reshaping (and also interpretation, see below the choice of edges) of data tables to graphs, we based on https://colab.research.google.com/drive/1_eR7DXBF3V4EwH946dDPOxeclDBeKNMD?usp=sharing#scrollTo=WuggdIItffpv.

## General preparation - handle categorical features
Here, we handle the categorical features through label-encoding. 

In [2]:
import sys
sys.path.append("../")

import time
from sklearn import datasets

from sklearn.preprocessing import LabelEncoder

from torch_geometric.data import Data

import itertools
import numpy as np
import pandas as pd

from classif_basic.data_preparation import train_valid_test_split, set_target_if_feature, automatic_preprocessing

### Prepare data

Fix precise % of population distribution (sex: Male, Female) and % of wealthiness according to sex. In that way, we could inspect if the structure of the model (here based on a graph) integrates this "sexist" representation of the world. 

In [3]:
# preparing the dataset on clients for binary classification
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1590, as_frame=True)

t0 = time.time()

X = data.data
Y = (data.target == '>50K') * 1

In [4]:
dataset = X.copy()
dataset['target'] = Y
dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,0
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,0
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,1
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,1
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,0
48838,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,1
48839,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,0
48840,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,0


In [5]:
# here, "treatment" is saw as being 'Male' and not 'Female'

df_response_if_feature=dataset.loc[(dataset['sex']=='Male')&(dataset['target']==1)]
df_no_response_if_feature=dataset.loc[(dataset['sex']=='Male')&(dataset['target']==0)]
df_response_if_not_feature=dataset.loc[(dataset['sex']=='Female')&(dataset['target']==1)]
df_no_response_if_not_feature=dataset.loc[(dataset['sex']=='Female')&(dataset['target']==0)]

print(df_response_if_feature.shape[0])
print(df_no_response_if_feature.shape[0])
print(df_response_if_not_feature.shape[0])
print(df_no_response_if_not_feature.shape[0])


# % of men selected by the initial data
df_response_if_feature.shape[0]/(df_response_if_feature.shape[0]+df_no_response_if_feature.shape[0])

9918
22732
1769
14423


0.3037672281776417

In [6]:
# % of women selected by the initial data
df_response_if_not_feature.shape[0]/(df_response_if_feature.shape[0]+df_no_response_if_not_feature.shape[0])

0.07267573230352081

In [7]:
len_dataset = 20_000

percentage_feature= 70
percentage_response_if_feature=70
percentage_response_if_not_feature=10

sexist_dataset = set_target_if_feature(
    df_response_if_feature=df_response_if_feature,
    df_no_response_if_feature=df_no_response_if_feature,
    df_response_if_not_feature=df_response_if_not_feature,
    df_no_response_if_not_feature=df_no_response_if_not_feature,
    len_dataset=len_dataset,
    percentage_feature=percentage_feature,
    percentage_response_if_feature=percentage_response_if_feature,
    percentage_response_if_not_feature=percentage_response_if_not_feature)

len_dataset: 20000
nb indivs feature with response: 9800
nb indivs feature with no response: 4200
nb indivs not_feature with response: 600
nb indivs not_feature with no response: 5400


In [8]:
X = sexist_dataset.loc[: , dataset.columns != 'target']
Y = sexist_dataset['target']

In [9]:
Y

47229    1
44097    1
9780     1
44824    1
17676    1
        ..
42335    0
24628    0
7618     0
2804     0
7250     0
Name: target, Length: 20000, dtype: int64

## Reshape (by interpreting) data to a graph

From this dataset (where we introduced selectively a "sexist" effect against women), let's see how we could swith from the tabular data to a graph representation.

The point is that our features X all seem to be attributes of the clients, though we should find a way of representing their interactions between clients 

X = {race, age, sex, final weight (depends on age, sex, hispanic origin, race), education, education number, marital status, relationship, occupation, hours per week, workclass, race, sex, capital gain, capital loss, native country} 

**Nodes** 
Bank clients (by ID)

**Edges** 
Here, we should find one or several ways of connecting the clients

Should be occupation → if changes of occupation (or similar client with new occupation), which impact on the revenue? // change of football team => impact on the football rate 
(pers) actionable => predict revenue when switches to a new job??
→ may be: “hours per week” <=> inspect the change of revenue if switches to greater hours per week?

**Node Features** 
Attributs of the nodes, i.e. characteristics of the clients (here, hard to separate from what "connects" them...) 

Race, age, sex, final weight (depends on age, sex, hispanic origin, race), education, education number, marital status, relationship, hours per week, workclass, race, sex, capital gain, capital loss, native country 

**Label (here at a node-level?)** 
Income (Y = income > $50 000)

In [10]:
# first of all, specify the edge
edge = "occupation"# str (for the moment)

In [11]:
#Make sure that we have no duplicate nodes
X.index.unique().shape[0] == X.shape[0]

True

**Extract the node features**

The node features are typically represented in a matrix of the shape (num_nodes, node_feature_dim).

For each of the bank clients, we simply extract their attributes (except here the "occupation", that would be used as an "actionable" edge to connect them)

In [12]:
node_features = X.loc[:, X.columns != edge]
node_features

,age,workclass,fnlwgt,education,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
47229,41.0,Private,433989.0,Assoc-voc,11.0,Married-civ-spouse,Husband,White,Male,4386.0,0.0,60.0,United-States
44097,33.0,Self-emp-not-inc,108438.0,HS-grad,9.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,40.0,United-States
9780,46.0,Local-gov,200727.0,Masters,14.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,50.0,United-States
44824,68.0,NaN,255276.0,Some-college,10.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,48.0,United-States
17676,42.0,Private,230624.0,10th,6.0,Never-married,Unmarried,White,Male,0.0,0.0,40.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42335,43.0,Private,191765.0,HS-grad,9.0,Divorced,Unmarried,Black,Female,0.0,0.0,35.0,United-States
24628,45.0,Local-gov,203322.0,Some-college,10.0,Divorced,Not-in-family,White,Female,0.0,0.0,38.0,United-States
7618,37.0,Private,216129.0,HS-grad,9.0,Never-married,Not-in-family,Black,Female,0.0,0.0,45.0,United-States
2804,21.0,NaN,34446.0,Some-college,10.0,Never-married,Own-child,White,Female,0.0,0.0,35.0,United-States


That's already our node feature matrix. The number of nodes and the ordering is implicitly defined by it's shape. Each row corresponds to one node in our final graph. 

In [13]:
# Convert to numpy
x = node_features.to_numpy()
x.shape # [num_nodes x num_features]

(20000, 13)

**Extract the labels**

Those are simply the wealthiness of each of the clients (if their income is >$50 000). This corresponds to a node-level prediction problem. Therefore we have as many labels as we have nodes.

In [14]:
labels = Y
labels.head()

47229    1
44097    1
9780     1
44824    1
17676    1
Name: target, dtype: int64

In [15]:
# to make the graph functioning, check that the nodes follow the same order than the labels (rows n°)
# else, sort values by ids

nb_corresponding_nodes_labels = (labels.index == node_features.index).sum()

nb_corresponding_nodes_labels == X.shape[0]

True

In [16]:
# Convert to numpy
y = labels.to_numpy()
y.shape # [num_nodes, 1] --> node regression

(20000,)

**Extract the edges**

That's probably the trickiest part with a tabular dataset. You need to think of a reasonable way to connect your nodes. As mentioned previously, we will use the team assignment here.

    AGAIN: There are many ways to connect the entities in a dataset and this approach is very trivial (as it will lead to disconnected subgraphs). If I wanted to build a real model from this dataset, I would probably look for a more sophisticated way to connect the clients. Using a GNN is a bit overkill for the way I model the edges.

We now need to find the pairs of clients that are assigned to the same type of job. Let's first check how many clients per type of job we have.

In [17]:
# get an idea of the codes corresponding to occupations, reconstituting labels' transformations from X
le = LabelEncoder()

dict_occupation_codes = pd.Series(X[edge].values, index=X.apply(le.fit_transform)[edge]).to_dict()

# correct according to dict comparison
dict_occupation_codes[14] = 'Transport-moving'
dict_occupation_codes

{11: 'Sales',
 2: 'Craft-repair',
 9: 'Prof-specialty',
 14: 'Transport-moving',
 13: 'Transport-moving',
 3: 'Exec-managerial',
 4: 'Farming-fishing',
 10: 'Protective-serv',
 12: 'Tech-support',
 6: 'Machine-op-inspct',
 0: 'Adm-clerical',
 5: 'Handlers-cleaners',
 7: 'Other-service',
 1: 'Armed-Forces',
 8: 'Priv-house-serv'}

In [18]:
# With the profession types, this tells us how many clients per type of profession we have to connect
df_jobs = X.replace({"occupation": dict_occupation_codes})
df_jobs["occupation"].value_counts()

Exec-managerial      3418
Prof-specialty       3341
Sales                2467
Craft-repair         2237
Adm-clerical         2111
Other-service        1527
Machine-op-inspct     969
Transport-moving      832
Tech-support          690
Handlers-cleaners     513
Farming-fishing       452
Protective-serv       427
Priv-house-serv        85
Armed-Forces            7
Name: occupation, dtype: int64

We now need to build all permutations of these clients within one type of job, which corresponds to a fully-connected graph within each occupation-subgroup. We use the column int_player_id as indices for the edges. If there is for example a [0, 1] in the edge index, it means that the first and second node (regarding the previously defined node feature matrix) are connected.

In [19]:
jobs = X["occupation"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))
for job in jobs:
    job_df = X[X["occupation"] == job]
    clients = job_df.index
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(clients, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    clients_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, clients_edges])
# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]

array([[47229., 47229., 47229., ..., 45281., 45281., 15934.],
       [20141., 19337., 38997., ..., 15934., 14297., 14297.]])

The result are these source/target edge pairs. Here you can also model dircted or undirected edges by inluding both or just one direction (I included both). This COO format is usually chosen as it is more efficient than a NxN adjacency matrix.

**To Do then: include ONE-SENSE direction** for certain features (against non-sense)

**Final step - build the graph dataset**

Now we have all the components we need to build a graph for libraries like Pytorch Geometric or DGL. 

We need to pass the numpy arrays to the Data object, like this. If you have further attributes like edge_features, you can also pass them here.

(pers) work hours and sector --> edge features?

In [20]:
data = Data(x=x, edge_index=edge_index, y=y)

In [21]:
data

Data(x=[20000, 13], edge_index=[2, 21734299], y=[20000])

# Train a basic Graph Neural Network on the graph-shaped data

## Training a Graph Neural Network (GNN)

We can easily convert our MLP to a GNN by swapping the `torch.nn.Linear` layers with PyG's GNN operators.

Following-up on [the first part of the Torch tutorial we used](https://colab.research.google.com/drive/1h3-vJGRVloF5zStxL5I0rSy4ZUPNsjy8), we replace the linear layers by the [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv) module.
To recap, the **GCN layer** ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)) is defined as

$$
\mathbf{x}_v^{(\ell + 1)} = \mathbf{W}^{(\ell + 1)} \sum_{w \in \mathcal{N}(v) \, \cup \, \{ v \}} \frac{1}{c_{w,v}} \cdot \mathbf{x}_w^{(\ell)}
$$

where $\mathbf{W}^{(\ell + 1)}$ denotes a trainable weight matrix of shape `[num_output_features, num_input_features]` and $c_{w,v}$ refers to a fixed normalization coefficient for each edge.
In contrast, a single `Linear` layer is defined as

$$
\mathbf{x}_v^{(\ell + 1)} = \mathbf{W}^{(\ell + 1)} \mathbf{x}_v^{(\ell)}
$$

which does not make use of neighboring node information.

In [29]:
import torch
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(data.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2) # number of classes on the data

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(13, 16)
  (conv2): GCNConv(16, 2)
)


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
    return test_acc


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

### Train-test-split? Or special with np vectors (see Torch implementation)?

In [ ]:
X_train, X_valid, X_train_valid, X_test, Y_train, Y_valid, Y_train_valid, Y_test = train_valid_test_split(
    X=X,
    Y=Y, 
    model_task=model_task,
    preprocessing_cat_features=preprocessing_cat_features)

In [ ]:
augmented_train_valid_set = augment_train_valid_set_with_results("uncorrected", X_train_valid, Y_train_valid, Y_pred_train_valid, model_task)

We now see that this process with basic data preparation, modelling and integration of the results in a DataFrame (as storage of the model) is very fast (in seconds):

In [ ]:
t1 = time.time()

print(f"Basic modelling took {round(t1 - t0)} seconds")

The further steps are for fairness assessment and correction of the model, functionality which is available with the package FairDream of DreamQuark (private for the moment)...

## Detection alert (on train&valid data to examine if the model learned discriminant behavior)

## Discrimination correction with a new fair model

### Generating fairer models with grid search or weights distorsion

### Evaluating the best fair model